In [1]:
import os
from tqdm import tqdm

# torch imports
import torch
import torch.nn as nn
import torch.optim as optim

# helper functions for computer vision
import torchvision
import torchvision.transforms as transforms

In [ ]:
import random

In [22]:
import numpy as np

In [34]:
# class adapted from code developed for CS 540 Spring 2023 HW 7

class LeNet(nn.Module):
    def __init__(self, input_shape=(32, 32), num_classes=100):
        super(LeNet, self).__init__()

        # TODO maybe disable bias? 

        self.step1 = nn.Linear(in_features=2, out_features=2)
        self.step2 = nn.Sigmoid()
        self.step3 = nn.Linear(in_features=2, out_features=1)


    def forward(self, x):

        # RUN IT ON A GPU if it exists
        if torch.cuda.is_available():
            x = x.to("cuda")

        print("Red flag")
        print(type(x))
        print(x)

        # layer 1
        out1 = self.step1(x)

        # layer 2
        out2 = self.step2(out1)

        # layer 3
        out3 = self.step3(out2)

        # adapted from https://stackoverflow.com/questions/46826218/pytorch-how-to-get-the-shape-of-a-tensor-as-a-list-of-int
        #shape_dict[1] = list(out.shape)

        return out3


def count_model_params():
    '''
    return the number of trainable parameters of LeNet.
    '''
    model = LeNet()
    model_params = 0.0
    for _, parameter in model.named_parameters():
        if (parameter.requires_grad):
            model_params += parameter.numel()
    return model_params


def train_model(model, random_pairs, first_is_larger, epoch):
    """
    model (torch.nn.module): The model created to train
    train_loader (pytorch data loader): Training data loader
    optimizer (optimizer.*): A instance of some sort of optimizer, usually SGD
    criterion (nn.CrossEntropyLoss) : Loss function used to train the network
    epoch (int): Current epoch number
    """

    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    criterion = nn.CrossEntropyLoss()

    model.train()
    train_loss = 0.0
    for i in range(1, len(random_pairs)):
        input = random_pairs[i]
        target = first_is_larger[i]

        # 1) zero the parameter gradients
        optimizer.zero_grad()
        # 2) forward + backward + optimize
        output, _ = model(input)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        # Update the train_loss variable
        # .item() detaches the node from the computational graph
        train_loss += loss.item()

    train_loss /= len(random_pairs)
    print('[Training set] Epoch: {:d}, Average loss: {:.4f}'.format(epoch+1, train_loss))

    return train_loss


def test_model(model, test_loader, epoch):
    model.eval()
    correct = 0
    with torch.no_grad():
        for input, target in test_loader:
            output, _ = model(input)
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_acc = correct / len(test_loader.dataset)
    print('[Test set] Epoch: {:d}, Accuracy: {:.2f}%\n'.format(
        epoch+1, 100. * test_acc))

    return test_acc



In [35]:
myModel = LeNet()
print(myModel)
print(count_model_params())

LeNet(
  (step1): Linear(in_features=2, out_features=2, bias=True)
  (step2): Sigmoid()
  (step3): Linear(in_features=2, out_features=1, bias=True)
)
9.0


In [36]:
# Generate some pairs of random integers
# Adapted from ChatGPT


# Define the range for your random integers
min_value = 1
max_value = 100

# Generate random pairs of integers
num_pairs = 500
random_pairs = [[random.randint(min_value, max_value), random.randint(min_value, max_value)] for _ in range(num_pairs)]

# Print the generated pairs
for pair in random_pairs:
    print(pair)

first_is_larger = []
for pair in random_pairs: 
    first_is_larger.append(int(pair[0] >= pair[1]))


[93, 54]
[90, 44]
[89, 40]
[46, 45]
[49, 39]
[95, 51]
[97, 17]
[95, 4]
[78, 60]
[6, 76]
[25, 12]
[67, 62]
[2, 90]
[16, 46]
[46, 36]
[36, 20]
[40, 31]
[70, 36]
[61, 62]
[14, 93]
[84, 74]
[37, 80]
[90, 15]
[39, 37]
[20, 61]
[93, 84]
[39, 33]
[77, 72]
[71, 31]
[74, 97]
[10, 97]
[10, 4]
[76, 80]
[23, 11]
[53, 82]
[11, 31]
[28, 51]
[38, 90]
[25, 69]
[12, 15]
[15, 8]
[69, 22]
[28, 23]
[80, 31]
[64, 81]
[97, 44]
[57, 43]
[12, 93]
[58, 60]
[29, 20]
[83, 18]
[11, 42]
[98, 63]
[53, 50]
[50, 46]
[96, 35]
[64, 92]
[74, 1]
[86, 4]
[80, 14]
[90, 84]
[94, 66]
[38, 95]
[94, 73]
[28, 44]
[44, 65]
[51, 48]
[26, 89]
[67, 82]
[25, 73]
[96, 46]
[36, 43]
[47, 94]
[18, 79]
[32, 40]
[77, 32]
[81, 40]
[14, 3]
[86, 59]
[100, 5]
[73, 13]
[60, 90]
[98, 16]
[44, 19]
[13, 73]
[69, 19]
[59, 97]
[35, 64]
[65, 74]
[15, 86]
[38, 82]
[35, 41]
[23, 53]
[3, 97]
[21, 42]
[96, 71]
[20, 27]
[12, 55]
[76, 14]
[31, 92]
[60, 98]
[18, 74]
[47, 20]
[35, 54]
[98, 16]
[1, 19]
[36, 38]
[87, 99]
[59, 70]
[38, 53]
[38, 42]
[46, 35]
[9

In [ ]:
for i in range(1, len(random_pairs)):
    input = random_pairs[i]
    target = first_is_larger[i]

In [37]:
# random_pairs = torch.tensor(random_pairs)
# first_is_larger = torch.tensor(first_is_larger)

train_model(model=myModel, random_pairs=torch.tensor(random_pairs), first_is_larger=torch.tensor(first_is_larger), epoch=1)

Red flag
<class 'torch.Tensor'>
tensor([90, 44])


RuntimeError: expected scalar type Long but found Float

In [25]:
type(random_pairs[1])

torch.Tensor

In [30]:
for i in range(1, len(random_pairs)):
    print(random_pairs[i][1])

tensor(15, dtype=torch.int32)
tensor(55, dtype=torch.int32)
tensor(9, dtype=torch.int32)
tensor(32, dtype=torch.int32)
tensor(94, dtype=torch.int32)
tensor(50, dtype=torch.int32)
tensor(53, dtype=torch.int32)
tensor(95, dtype=torch.int32)
tensor(31, dtype=torch.int32)
tensor(16, dtype=torch.int32)
tensor(75, dtype=torch.int32)
tensor(46, dtype=torch.int32)
tensor(3, dtype=torch.int32)
tensor(75, dtype=torch.int32)
tensor(95, dtype=torch.int32)
tensor(97, dtype=torch.int32)
tensor(29, dtype=torch.int32)
tensor(71, dtype=torch.int32)
tensor(22, dtype=torch.int32)
tensor(60, dtype=torch.int32)
tensor(44, dtype=torch.int32)
tensor(45, dtype=torch.int32)
tensor(20, dtype=torch.int32)
tensor(49, dtype=torch.int32)
tensor(33, dtype=torch.int32)
tensor(33, dtype=torch.int32)
tensor(74, dtype=torch.int32)
tensor(75, dtype=torch.int32)
tensor(87, dtype=torch.int32)
tensor(74, dtype=torch.int32)
tensor(68, dtype=torch.int32)
tensor(43, dtype=torch.int32)
tensor(64, dtype=torch.int32)
tensor(24, d